# Mixed effect model analysis

## Load data and packages

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import statsmodels.formula.api as smf

In [2]:
df = pd.read_excel("./data/df_for_analysis.xlsx",index_col=0)

In [3]:
def keep_weight_post_infection(x,weight_end = "weight_T14"):
    """
    Given a DataFrame `x`, returns a Series containing the weights post-infection.

    *Arguments*
    - x: DataFrame containing the data.
    - weight_end: Column name of the last weight measurement.

    *Returns*
    - shifted_series: Series containing the weights post-infection.
    """
    # Extract relevant columns from the DataFrame
    dates = x['Dates']
    t_infection = x['Time_infection']
    datas = x['weight_T_infection':weight_end]

    # Find the date closest to the infection time
    new_time_infection = dates[dates <= t_infection][-1]
    location_of_TI = dates.get_loc(new_time_infection)

    # Return the original series if the data at the infection time is NaN
    if np.isnan(datas[location_of_TI]):
        return datas
    
    # Shift the values of the input series by the specified index
    shifted_series = pd.Series([np.nan] * len(datas), index=datas.index)
    if location_of_TI == 0:
        return datas
    else:
        shifted_series[:-location_of_TI] = datas.values.tolist()[location_of_TI:]
    # Shift the values of the input series by the specified index
    return shifted_series

### Transform data to long format

Keep only data that are after the time of infeciton

In [4]:
# change dates column to datetimindex and transform weight datas to numeric only
df.loc[:,"weight_T_infection":"weight_T14"] = df.loc[:,"weight_T_infection":"weight_T14"].apply(pd.to_numeric,errors='coerce')
serie_dates = df['Time_point'].apply(lambda x: pd.to_datetime(x.split(','),dayfirst=True))
df['Dates'] = serie_dates
data = df.apply(lambda x: keep_weight_post_infection(x),axis=1)
data

C:\Users\ma7054\AppData\Local\Temp\ipykernel_14256\794712423.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,"weight_T_infection":"weight_T14"] = df.loc[:,"weight_T_infection":"weight_T14"].apply(pd.to_numeric,errors='coerce')


,weight_T_infection,weight_T1,weight_T2,weight_T3,weight_T4,weight_T5,weight_T6,weight_T7,weight_T8,weight_T9,weight_T10,weight_T11,weight_T12,weight_T13,weight_T14
0,23.92,21.72,20.96,19.38,18.16,16.44,15.49,15.44,15.05,NaN,NaN,NaN,NaN,NaN,NaN
1,21.40,19.45,18.84,17.82,16.80,15.02,14.14,14.40,14.73,NaN,NaN,NaN,NaN,NaN,NaN
2,22.56,21.45,20.83,18.67,16.82,15.30,14.74,14.80,14.42,NaN,NaN,NaN,NaN,NaN,NaN
3,20.39,18.69,16.60,15.58,14.17,13.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23.72,21.74,20.29,19.56,18.50,16.80,16.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,27.70,25.00,22.80,21.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2353,26.10,24.90,22.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2354,24.60,25.80,25.70,24.90,22.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2355,27.40,27.30,28.20,27.80,27.70,27.80,27.60,28.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Normalization by weight at T infection and replace the normalize data into the original dataframe

In [5]:

normalize = data.div(data['weight_T_infection'],axis=0)*100
df_normalize = df.copy()
df_normalize.loc[:,"weight_T_infection":"weight_T14"] = normalize

df_normalize['min_weight'] = df_normalize.loc[:,"weight_T_infection":"weight_T14"].min(axis=1)
df_normalize['t_origin'] = df_normalize['time_original']
df_normalize

,Mouse_ID,ID_Experiment,Cage,Strain,Date,Experiment,Group,Group_info,H0,Pre_traitment,...,survival_0.06,time_0.05,survival_0.05,time_original,survival_original,max_loss_weight_percentage,exp,sub_exp,min_weight,t_origin
0,TRO-05432,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,9.0,1,0.629181,1,A,62.918060,9.0
1,TRO-05433,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,9.0,1,0.660748,1,A,66.074766,9.0
2,TRO-05434,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,2.5,1,9.0,1,0.639184,1,A,63.918440,9.0
3,TRO-05435,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,6.0,1,0.664051,1,A,66.405101,6.0
4,TRO-05456,ID_001,B,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,7.0,1,0.707420,1,A,70.741990,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,TRO-028337,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,1.5,1,5.0,1,0.761733,3,no,76.173285,5.0
2353,TRO-028338,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,2.5,1,4.0,1,0.865900,3,no,86.590038,4.0
2354,TRO-028339,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,5.5,1,6.0,1,0.926829,3,no,92.682927,6.0
2355,TRO-028342,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,0,11.0,0,11.0,0,0.996350,3,no,99.635036,11.0


Find only releavent columns


In [6]:
columns = df_normalize.loc[:,"weight_T_infection":"weight_T14"].columns.tolist()
columns_index = df_normalize.loc[:, ~df_normalize.columns.isin(columns)]
column_time = [n for n in df_normalize.columns.tolist() if "time_" in n]
column_time = [column_time[-1]] + column_time[:-1]
columns_index_time = ['ID_Experiment','Mouse_ID','Date','Infection','Group','exp','survival_original','t_origin']

Transform to tidy data for releavent columns

In [33]:
df_longer_weight = df_normalize.melt(id_vars=columns_index_time,value_vars=columns,var_name="Time",value_name="weight")
df_longer_weight['Time'] = df_longer_weight['Time'].apply(lambda x: "".join(x.split("_")[1:]))
df_longer_weight

,ID_Experiment,Mouse_ID,Date,Infection,Group,exp,survival_original,t_origin,Time,weight
0,ID_001,TRO-05432,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
1,ID_001,TRO-05433,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
2,ID_001,TRO-05434,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
3,ID_001,TRO-05435,2014-06-05,C. albicans,1A,1,1,6.0,Tinfection,100.0
4,ID_001,TRO-05456,2014-06-05,C. albicans,1A,1,1,7.0,Tinfection,100.0
...,...,...,...,...,...,...,...,...,...,...
35350,ID_096,TRO-028337,2023-03-03,S. pneumoniae,3,3,1,5.0,T14,NaN
35351,ID_096,TRO-028338,2023-03-03,S. pneumoniae,3,3,1,4.0,T14,NaN
35352,ID_096,TRO-028339,2023-03-03,S. pneumoniae,3,3,1,6.0,T14,NaN
35353,ID_096,TRO-028342,2023-03-03,S. pneumoniae,3,3,0,11.0,T14,NaN


In [129]:
df_dead = df_longer_weight[df_longer_weight['survival_original']==1]
df_survive = df_longer_weight[df_longer_weight['survival_original']==0]
df_dead_T6 = df_dead[df_dead['Time'] == 'T6']

In [94]:
stats = df_dead_T6.groupby(['Infection'])['weight'].agg(['mean','median','count','std'])
stats['ci95_hi'] = stats['median'] + 1.96*stats['std']/np.sqrt(stats['count'])
stats['ci95_low'] = stats['median'] - 1.96*stats['std']/np.sqrt(stats['count'])
stats


,mean,median,count,std,ci95_hi,ci95_low
Infection,,,,,,
C. albicans,73.499218,71.938776,59,8.044081,73.991386,69.886165
H1N1,75.780044,74.025974,83,7.659599,75.673844,72.378104
Listeria,77.476485,77.150065,12,5.384380,80.196565,74.103565
S. pneumoniae,91.174962,90.303030,75,8.396447,92.203325,88.402735


In [132]:
df_survive.groupby(["Infection",'Time'])['weight'].agg(['mean',"median","count","std"])

mean      median  count        std
Infection     Time                                                
C. albicans   T1           93.476163   94.154432    156   3.950844
              T10          91.554041   92.714520     94   8.823726
              T11          91.586372   92.911246     94   9.428057
              T12          91.637698   93.449706     94  10.015233
              T13          89.287552   92.455453     92   9.973504
              T14          92.858985   95.351161     34   8.220296
              T2           88.470998   89.800337    156   5.849797
              T3           86.800895   86.972064    156   8.048837
              T4           88.811919   88.431034    156   8.920442
              T5           90.715987   91.223746    156   9.366382
              T6           93.229003   95.405033    156   9.225846
              T7           90.612940   92.500926     94   7.709892
              T8           91.220919   92.754832     94   8.420971
              T9           89.183630   90.542887     94   8.696729
              Tinfection  100.000000  100.000000    156   0.000000
H1N1          T1           98.696077   99.416667    184   4.479213
              T10          94.067609   97.451520    132  11.496328
              T11          93.867819   98.439211    104  11.571646
              T12          98.184990  102.514765     20  10.523621
              T13         100.212825  102.150538      9   7.980974
              T14                NaN         NaN      0        NaN
              T2           96.171915   96.233108    184   5.904801
              T3           94.427212   95.361408    184   7.789269
              T4           92.716280   93.272097    184   9.217514
              T5           90.739214   91.484071    184  10.388433
              T6           89.301377   91.808680    168  12.141383
              T7           91.129323   95.100763    160  11.827109
              T8           92.801801   95.953757    155  11.792062
              T9           93.529833   97.818182    139  11.513178
              Tinfection  100.000000  100.000000    184   0.000000
Listeria      T1           99.045345   99.264507    460   3.573729
              T10          95.945149   98.152297     40  10.793708
              T11         103.749123  102.834704     16   4.050310
              T12         106.248397  107.113971     16   3.606188
              T13                NaN         NaN      0        NaN
              T14                NaN         NaN      0        NaN
              T2           93.675744   93.658537    457   5.615993
              T3           92.196297   93.193717    460   7.561325
              T4           92.792496   94.322917    434   8.760431
              T5           95.906214   97.058824    391   8.318443
              T6           96.157849   97.340426    283   8.211604
              T7           97.652866   98.598650    206   7.955902
              T8           97.766336   98.550725    153   7.316059
              T9           98.054770   99.024390     97   8.016770
              Tinfection  100.000000  100.000000    460   0.000000
S. pneumoniae T1           97.995365   98.081435    306   3.252254
              T10         100.296108  100.904977    111   4.980912
              T11         100.283809  101.020408     49   4.726532
              T12         100.261878  101.923077     41   5.584517
              T13         100.290374  100.816327      7   5.239686
              T14                NaN         NaN      0        NaN
              T2           99.107057   99.267064    308   3.579106
              T3           99.637596  100.000000    308   4.763233
              T4           99.159382   99.439666    288   3.757713
              T5           99.876227  100.433842    268   3.815621
              T6          100.526110  101.092896    259   4.175913
              T7          100.967623  101.408451    223   3.972189
              T8          100.919776  101.405222    212   4.381979
            

save the datas

In [41]:
stats.groupby(['Infection']).min()

,mean,median,count,std,ci95_hi,ci95_low
Infection,,,,,,
C. albicans,73.499218,71.938776,23,0.0,73.991386,69.886165
H1N1,71.973942,72.186570,18,0.0,73.428070,70.945071
Listeria,72.995781,72.995781,0,0.0,78.416559,71.977986
S. pneumoniae,83.044541,81.621912,8,0.0,85.436483,77.807340


In [58]:
df_longer_weight.to_excel("./data/df_long_format_for_analysis.xlsx")

## Mixed effect model
### function

In [118]:
def Mixed_Effects_Models(df,chosen_infection = 'S. pneumoniae',time_to_exclude = 8):
    df_infection = df[df['Infection'] == chosen_infection]
    df_infection = df_infection[~df_infection['Time'].isin([f"T{n}" for n in range(time_to_exclude,15,1)])]#remove unused data
    
    time_point = df_infection['Time'].unique()
    weight_point_to_integer = dict(zip(time_point,[n for n in range(len(time_point))]))
    
    df_infection['Time'] = df_infection['Time'].replace(weight_point_to_integer)
    model = smf.mixedlm("weight ~ Time",df_infection,groups=df_infection['survival_original'],missing="drop").fit()
    return model.summary()

In [125]:
Mixed_Effects_Models(df_longer_weight,"S. pneumoniae",3)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 2122    Method:             REML      
No. Groups:       2       Scale:              15.4058   
Min. group size:  922     Log-Likelihood:     -5916.7902
Max. group size:  1200    Converged:          Yes       
Mean group size:  1061.0                                
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept   99.984    0.912 109.672 0.000 98.197 101.770
Time        -1.857    0.104 -17.839 0.000 -2.061  -1.653
Group Var    1.626    0.588                             
========================================================

"""

In [124]:
Mixed_Effects_Models(df_longer_weight,"Listeria",3)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 3104    Method:             REML      
No. Groups:       2       Scale:              13.2257   
Min. group size:  1377    Log-Likelihood:     -8417.4402
Max. group size:  1727    Converged:          Yes       
Mean group size:  1552.0                                
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept  100.967    1.490  67.746 0.000 98.046 103.888
Time        -4.888    0.080 -61.087 0.000 -5.045  -4.731
Group Var    4.421    1.713                             
========================================================

"""

In [126]:
Mixed_Effects_Models(df_longer_weight,"C. albicans",5)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 1199    Method:             REML      
No. Groups:       2       Scale:              38.0819   
Min. group size:  419     Log-Likelihood:     -3886.4988
Max. group size:  780     Converged:          Yes       
Mean group size:  599.5                                 
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept   97.546    1.719  56.736 0.000 94.176 100.915
Time        -3.866    0.126 -30.697 0.000 -4.112  -3.619
Group Var    5.724    1.326                             
========================================================

"""

In [127]:
Mixed_Effects_Models(df_longer_weight,"H1N1",5)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 1666    Method:             REML      
No. Groups:       2       Scale:              36.3636   
Min. group size:  746     Log-Likelihood:     -5361.3488
Max. group size:  920     Converged:          Yes       
Mean group size:  833.0                                 
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept  100.337    2.062  48.660 0.000 96.295 104.378
Time        -2.991    0.105 -28.504 0.000 -3.196  -2.785
Group Var    8.373    1.981                             
========================================================

"""